In [1]:
from xclingo import XclingoControl
xclingo_control = XclingoControl(
    ['0'],  # internal clingo control arguments, 0 = "all the models"
    n_explanations='0'  # Desired number of explanations per model, 0 = "all the explanations"
)

In [2]:
annotated_program = """
    person(gabriel;clare).

    drive(gabriel).
    alcohol(gabriel, 40).
    resist(gabriel).

    drive(clare).
    alcohol(clare, 5).

    %!trace_rule {"% drove drunk", P}.
    punish(P) :- drive(P), alcohol(P,A), A>30, person(P).

    %!trace_rule {"% resisted to authority", P}.
    punish(P) :- resist(P), person(P).

    %!trace_rule {"% goes to prison",P}.
    sentence(P, prison) :- punish(P).

    %!trace_rule {"% is innocent by default",P}.
    sentence(P, innocent) :- person(P), not punish(P).

    %!trace {alcohol(P,A), "% alcohol's level is %",P,A} :- alcohol(P,A).
    %!trace {alcohol(P,A), "% was drunk",P} :- alcohol(P,A).

    %!show_trace {sentence(P,S)} :- sentence(P,S).
"""

In [6]:
xclingo_control.add('base', [], annotated_program)
xclingo_control.ground([("base", [])])

#### xclingo models (XclingoModel)
Calling solve on xclingo_control will yield `XclingoModel` objects.
This object work as the usual `Model` class from clingo (it is, in fact, an extension to that class), but has an additional method to compute (and yield) the explanation graphs from that especific model: `explain_model()`.
The objects yielded belong to the class `ExplanationGraphModel`.

‼️‼️ Important: The explanation graphs will only be computed in the case you ask for one via calling `explain_model()`. The computing of the graphs is asynchronous, next explanation graph won't be computed until user asks for it.

#### explanation graphs (ExplanationGraphModel)
The objects yielded by `explain_model()`.
The class is also an extension of clingo `Model`, and so it works as a usual model.
However, the symbols in it represent the explanation of the model, and not the symbols from the model itself.
The object additionally has a method `explain(self, symbol: Symbol)`, which retrieves the local explanation (object from `Explanation` class) for a symbol.
The symbol the explanation is requested for has to be a node of the graph (in other words, it has to be true for the model being explained).

#### explanations for a symbol (Explanation)
`Explanation` is a tree-like object, where each object (node in the tree) may have from zero to many child explanations.
Child nodes are accessed trough the `.causes` property.
Each node has a `.labels` property, containing the text labels associated with the symbol for that explanation.

In [ ]:
###### Example: the actual xclingo code for explaining SATISFIABLE programs.
nmodel = 0
for x_model in xclingo_control.solve():
    nmodel += 1
    print(f"Answer: {nmodel}")
    print(x_model)  # Prints a list of the true atoms for the model. Just like a usual Model object.
    nexpl = 0
    for graph_model in x_model.explain_model():
        nexpl += 1
        print(f"##Explanation: {nmodel}.{nexpl}")
        for sym in graph_model.show_trace:
            explanation = graph_model.explain(sym)
            if explanation is not None:  # A symbol could have no explanation.
                print(explanation)
    print(f"##Total Explanations:\t{nexpl}")
if nmodel > 0:
    print(f"Models:\t{nmodel}")


By default, the explanation will be presented as an ascii tree-like text, when printed.
However, the objects have several methods to freely modify the explanation or to exploit it.
Methods listed below:
```python
    def preorder_iterator(self, only_labelled=False) # Returns a “preorder” iterator over the explanation tree.
    def ascii_tree(self) # Returns a tree-like string depicting the explanation.
    def get_node_text(self) # “Gets” the text associated with a symbol for that explanation. Using all the labels.
    def add_cause(self, cause) # “Adds” a new child explanation node for the current node.
    def add_label(self, label) # “Adds” a new text label for the current symbol for that explanation.
```

As a very simple example, lets automatically translate the text in the labels

In [3]:
from xclingo.explanation import Explanation
from googletrans import Translator  # python3 -m pip install googletrans==3.1.0a0

def translate_explanation(explanation: Explanation, destiny_lang="es"):
    translator = Translator()
    for expl, depth_level in explanation.preorder_iterator(only_labelled=True):  # For each node, it replaces the labels with the translated text.
        expl.labels = set([translator.translate(label, dest=destiny_lang).text for label in expl.labels])

In [4]:
xclingo_control = XclingoControl(
    ['0'],  # internal clingo control arguments, 0 = "all the models"
    n_explanations='0'  # Desired number of explanations per model, 0 = "all the explanations"
)
xclingo_control.add('base', [], annotated_program)
xclingo_control.ground([("base", [])])

nmodel = 0
for x_model in xclingo_control.solve():
    nmodel += 1
    print(f"Answer: {nmodel}")
    print(x_model)  # Prints a list of the true atoms for the model. Just like a usual Model object.
    nexpl = 0
    for graph_model in x_model.explain_model():
        nexpl += 1
        print(f"##Explanation: {nmodel}.{nexpl}")
        for sym in graph_model.show_trace:
            explanation = graph_model.explain(sym)
            translate_explanation(explanation, destiny_lang="es")
            if explanation is not None:  # A symbol could have no explanation.
                print(explanation)
    print(f"##Total Explanations:\t{nexpl}")
if nmodel > 0:
    print(f"Models:\t{nmodel}")

Answer: 1
person(gabriel) person(clare) alcohol(gabriel,40) alcohol(clare,5) drive(gabriel) drive(clare) punish(gabriel) resist(gabriel) sentence(clare,innocent) sentence(gabriel,prison)
##Explanation: 1.1
  *
  |__"clare es inocente por defecto"

  *
  |__"gabriel va a la carcel"
  |  |__"Gabriel condujo borracho"
  |  |  |__"el nivel de alcohol de gabriel es 40";"gabriel estaba borracho"

##Explanation: 1.2
  *
  |__"clare es inocente por defecto"

  *
  |__"gabriel va a la carcel"
  |  |__"Gabriel se resistió a la autoridad"

##Total Explanations:	2
Models:	1


: 

#### 